In [2]:
from datasets import load_dataset

def load_verigui():
    ds_dict = load_dataset("2077AIDataFoundation/VeriGUI")
    # Use the only available split: "test"
    ds = ds_dict["test"]
    print(f"Loaded VeriGUI with {len(ds)} tasks (split='test').")
    return ds

ds = load_verigui()
ds[0]  # preview first example


c:\Users\hala\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded VeriGUI with 302 tasks (split='test').


{'folder': '675',
 'instruct': 'Find the Italian film that won the Grand Prix du Jury at the 1989 Cannes Film Festival, and was written and directed by Giuseppe Tornatore. For this film, please provide the following information: the name of the actor who played the blind projectionist Alfredo and won the BAFTA Award for Best Actor for this role; a famous quote about "life" spoken by this actor in the film; the specific award jointly received by the film\'s composer and his son; the duration of the film\'s longest version; the film\'s initial release date in Italy; the name of the designer who won the César Award for Best Poster; and the film\'s box office gross in the United States and Canada.',
 'result': ["film's title:Cinema Paradiso",
  "the actor's name:Philippe Noiret",
  'line spoken:Life isn’t like in the movies. Life… is much harder',
  'the names of the composer and his son:Ennio Morricone,Andrea Morricone,specific award:BAFTA Award for For Best Original Film Music',
  'durat

In [16]:
# # Map agent names to integer IDs
# AGENT_IDS = {"nav_agent": 0, "pers_agent": 1, "llm_agent": 2}

# def infer_agent_for_subtask(sub_instruction):
#     """Heuristic agent assignment based on subtask instruction"""
#     inst = sub_instruction.lower()
#     if any(k in inst for k in ['search', 'find', 'look up', 'navigate', 'click', 'scroll']):
#         return 'nav_agent'
#     if any(k in inst for k in ['filter', 'analyze', 'compare', 'extract', 'identify', 'plan']):
#         return 'llm_agent'
#     if any(k in inst for k in ['suggest', 'recommend', 'adapt', 'personalize']):
#         return 'pers_agent'
#     # default alternating
#     return 'llm_agent'

# def create_numeric_state(task_instruction, completed_subtasks, current_subtask, step_idx, total_steps):
#     """Create a numeric state vector"""
#     state = []

#     # Progress
#     progress = step_idx / total_steps if total_steps > 0 else 0.0
#     state.append(progress)

#     # Task complexity
#     complexity = min(len(completed_subtasks)/10.0, 1.0)
#     state.append(complexity)

#     # Instruction length
#     instr_len = min(len(task_instruction)/500.0, 1.0)
#     state.append(instr_len)

#     # Current subtask length
#     sub_len = min(len(current_subtask)/200.0, 1.0)
#     state.append(sub_len)

#     # Completed subtasks count (normalized)
#     state.append(min(len(completed_subtasks)/10.0,1.0))

#     # Available agents (all initially available)
#     state.extend([1.0, 1.0, 1.0])  # nav, pers, llm

#     # Step number normalized
#     state.append(min(step_idx/20.0, 1.0))

#     # Pad to fixed length
#     while len(state) < 16:
#         state.append(0.0)

#     return state[:16]

# def calculate_step_reward(step_idx, total_steps, agent):
#     """Shaped reward for RL"""
#     reward = 0.1  # small base reward

#     if step_idx == 0 and agent == 'llm_agent':
#         reward += 0.2
#     if step_idx == total_steps - 1 and agent == 'pers_agent':
#         reward += 0.2
#     if step_idx > 1:
#         reward -= 0.05  # penalize repeated steps

#     return max(reward, 0.01)

# def calculate_returns_to_go(rewards, gamma=0.99):
#     """Compute returns-to-go"""
#     returns = []
#     g = 0.0
#     for r in reversed(rewards):
#         g = r + gamma * g
#         returns.insert(0, g)
#     return returns


In [17]:
# import json

# def prepare_rl_transitions(ds):
#     transitions = []

#     for item in ds:
#         task_instruction = item["instruct"]
#         subtasks = item["sub_tasks"]
#         total_steps = len(subtasks)

#         for i, st in enumerate(subtasks):
#             current_instruction = st["instruct"]
#             agent_name = infer_agent_for_subtask(current_instruction)
#             action_id = AGENT_IDS[agent_name]

#             numeric_state = create_numeric_state(
#                 task_instruction, [s["result"] for s in subtasks[:i]], current_instruction, i, total_steps
#             )

#             reward = calculate_step_reward(i, total_steps, agent_name)
#             done = (i == total_steps - 1)

#             transition = {
#                 "state": numeric_state,
#                 "task_instruction": task_instruction,
#                 "current_subtask_instruction": current_instruction,
#                 "completed_subtasks": [s["result"] for s in subtasks[:i]],
#                 "action": action_id,
#                 "expected_output": st["result"],
#                 "reward": reward,
#                 "done": done
#             }

#             transitions.append(transition)

#     print("Prepared", len(transitions), "RL transitions with numeric states and shaped rewards.")
#     return transitions

# rl_data = prepare_rl_transitions(ds)
# rl_data[:2]  # preview first 2 transitions


In [ ]:
# import json
# import random

# # ============================================================
# # CONFIG
# # ============================================================

# AGENT_IDS = {
#     "nav_agent": 0,
#     "pers_agent": 1,
#     "llm_agent": 2
# }

# NUM_AGENTS = len(AGENT_IDS)
# STATE_DIM = 16


# # ============================================================
# # STATE CONSTRUCTION (NO ORACLE SIGNALS)
# # ============================================================

# def create_numeric_state(
#     task_instruction,
#     completed_subtasks,
#     current_subtask_instruction,
#     step_idx,
#     total_steps
# ):
#     """
#     Create an RL observation vector WITHOUT leaking answers.
#     """

#     state = []

#     # Progress through task (allowed)
#     state.append(step_idx / max(total_steps, 1))

#     # Number of completed subtasks (normalized)
#     state.append(len(completed_subtasks) / max(total_steps, 1))

#     # Task instruction length (proxy for difficulty)
#     state.append(min(len(task_instruction) / 500.0, 1.0))

#     # Current subtask length
#     state.append(min(len(current_subtask_instruction) / 200.0, 1.0))

#     # Total number of subtasks (normalized)
#     state.append(min(total_steps / 10.0, 1.0))

#     # Agent availability (all available by default)
#     state.extend([1.0] * NUM_AGENTS)

#     # Remaining steps ratio
#     remaining = (total_steps - step_idx - 1) / max(total_steps, 1)
#     state.append(max(remaining, 0.0))

#     # Padding
#     while len(state) < STATE_DIM:
#         state.append(0.0)

#     return state[:STATE_DIM]


# # ============================================================
# # DATASET PREPARATION (NO ACTIONS, NO REWARDS)
# # ============================================================

# def prepare_rl_dataset(raw_dataset):
#     """
#     Prepare dataset for RL environment consumption.

#     Output format:
#     Each entry represents a decision point.
#     """

#     samples = []

#     for item in raw_dataset:
#         task_instruction = item["instruct"]
#         subtasks = item["sub_tasks"]
#         total_steps = len(subtasks)

#         completed_results = []

#         for step_idx, sub in enumerate(subtasks):

#             obs = create_numeric_state(
#                 task_instruction=task_instruction,
#                 completed_subtasks=completed_results,
#                 current_subtask_instruction=sub["instruct"],
#                 step_idx=step_idx,
#                 total_steps=total_steps
#             )

#             sample = {
#                 # Observation
#                 "state": obs,

#                 # Text context (for env / embedding model)
#                 "task_instruction": task_instruction,
#                 "current_subtask_instruction": sub["instruct"],
#                 "completed_subtasks": completed_results.copy(),

#                 # Environment control signals
#                 "step_index": step_idx,
#                 "total_steps": total_steps,
#                 "done": (step_idx == total_steps - 1),

#                 # Ground truth execution result (NOT used for reward here)
#                 "expected_output": sub["result"],

#                 # Success flag placeholder (computed by environment)
#                 "success": None
#             }

#             samples.append(sample)
#             completed_results.append(sub["result"])

#     print(f"Prepared {len(samples)} RL decision points.")
#     return samples


# # ============================================================
# # OPTIONAL: TRAIN / VAL SPLIT
# # ============================================================

# def split_dataset(samples, val_ratio=0.1, seed=42):
#     random.seed(seed)
#     random.shuffle(samples)

#     split_idx = int(len(samples) * (1 - val_ratio))
#     return samples[:split_idx], samples[split_idx:]


# # ============================================================
# # USAGE
# # ============================================================

# # rl_samples = prepare_rl_dataset(ds)
# # train_data, val_data = split_dataset(rl_samples)

# # ============================================================
# # USAGE & SAVE TO FILE
# # ============================================================

# # If your dataset is already loaded in memory as `ds`, no need to read a file
# # ds = ...  # your dataset in memory

# # Prepare RL dataset
# rl_samples = prepare_rl_dataset(ds)

# # Optional: train/validation split
# train_data, val_data = split_dataset(rl_samples, val_ratio=0.1)

# # Save full RL dataset
# OUTPUT_FILE_ALL = "verigui_coordinator_rlnew.json"
# with open(OUTPUT_FILE_ALL, "w", encoding="utf-8") as f:
#     json.dump(rl_samples, f, indent=2, ensure_ascii=False)
# print(f"✅ Saved full RL dataset to: {OUTPUT_FILE_ALL}")

# # Save train/validation splits
# OUTPUT_FILE_TRAIN = "verigui_coordinator_rl_train.json"
# OUTPUT_FILE_VAL = "verigui_coordinator_rl_val.json"

# with open(OUTPUT_FILE_TRAIN, "w", encoding="utf-8") as f:
#     json.dump(train_data, f, indent=2, ensure_ascii=False)
# with open(OUTPUT_FILE_VAL, "w", encoding="utf-8") as f:
#     json.dump(val_data, f, indent=2, ensure_ascii=False)

# print(f"✅ Saved train split to: {OUTPUT_FILE_TRAIN}")
# print(f"✅ Saved validation split to: {OUTPUT_FILE_VAL}")


Prepared 1300 RL decision points.
✅ Saved full RL dataset to: verigui_coordinator_rlnew.json
✅ Saved train split to: verigui_coordinator_rl_train.json
✅ Saved validation split to: verigui_coordinator_rl_val.json


In [19]:
import json

OUTPUT_FILE = "verigui_coordinator_rlnew.json"

# Save only if we have data
if rl_samples and len(rl_samples) > 0:
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(rl_samples, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved RL-ready dataset to: {OUTPUT_FILE}")
else:
    print("⚠️ No RL samples to save! Check your dataset preprocessing.")


✅ Saved RL-ready dataset to: verigui_coordinator_rlnew.json


In [20]:
# OUTPUT_FILE = "verigui_coordinator_rlnew.json"

# with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
#     json.dump(rl_data, f, indent=2, ensure_ascii=False)

# print("✅ Saved RL-ready dataset to:", OUTPUT_FILE)


claude modifications: 15/1/2025  12:47 am

In [3]:
import json
import random

# ============================================================
# CONFIG
# ============================================================

GAMMA = 0.99
MAX_ACTION_ID = 12
STATE_DIM = 12

AGENT_IDS = {
    "nav_agent": 0,
    "pers_agent": 1,
    "llm_agent": 2
}
NUM_AGENTS = len(AGENT_IDS)

# ============================================================
# ACTION SPACE
# ============================================================

ACTION_SPACE = {
    0: "search_web",
    1: "search_database",
    2: "compute_statistic",
    3: "filter_data",
    4: "aggregate_results",
    5: "compare_values",
    6: "identify_maximum",
    7: "identify_minimum",
    8: "extract_entity",
    9: "verify_source",
    10: "summarize",
    11: "reason",
    12: "final_answer"
}

def sample_action(exploration=False):
    if exploration:
        return random.randint(0, MAX_ACTION_ID)
    return random.choice(list(ACTION_SPACE.keys()))

# ============================================================
# STATE CONSTRUCTION (PRUNED + TEMPORAL)
# ============================================================

def create_numeric_state(
    task_instruction,
    completed_subtasks,
    current_subtask_instruction,
    step_idx,
    total_steps
):
    state = []

    # Temporal progress
    state.append(step_idx / max(total_steps, 1))
    state.append((total_steps - step_idx - 1) / max(total_steps, 1))

    # Completion ratio
    state.append(len(completed_subtasks) / max(total_steps, 1))

    # Difficulty proxies
    state.append(min(len(task_instruction) / 500.0, 1.0))
    state.append(min(len(current_subtask_instruction) / 200.0, 1.0))

    # Agent availability
    state.extend([1.0] * NUM_AGENTS)

    # Padding
    while len(state) < STATE_DIM:
        state.append(0.0)

    return state[:STATE_DIM]

# ============================================================
# REWARD FUNCTION (DENSE + TERMINAL)
# ============================================================

def compute_reward(prev_progress, new_progress, done, success):
    reward = 0.0

    # Progress shaping
    reward += (new_progress - prev_progress)

    # Subtask completion bonus
    if new_progress > prev_progress:
        reward += 0.1

    # Terminal reward
    if done:
        reward += 1.0 if success else -1.0

    return reward

# ============================================================
# DATASET PREPARATION (FULL RL TRANSITIONS)
# ============================================================

def prepare_rl_dataset(raw_dataset, exploration_ratio=0.3):
    transitions = []
    episode_id = 0

    for task_id, item in enumerate(raw_dataset):
        task_instruction = item["instruct"]
        subtasks = item["sub_tasks"]
        total_steps = len(subtasks)

        completed_results = []
        prev_state = None
        prev_progress = 0.0
        episode_buffer = []

        for step_idx, sub in enumerate(subtasks):
            state = create_numeric_state(
                task_instruction,
                completed_results,
                sub["instruct"],
                step_idx,
                total_steps
            )

            action = sample_action(
                exploration=(random.random() < exploration_ratio)
            )

            progress = step_idx / max(total_steps, 1)
            done = (step_idx == total_steps - 1)

            # Offline assumption: last step = success
            success = done

            reward = compute_reward(
                prev_progress,
                progress,
                done,
                success
            )

            if prev_state is not None:
                episode_buffer.append({
                    # -------- IDs --------
                    "task_id": task_id,
                    "episode_id": episode_id,
                    "step_id": step_idx,

                    # -------- RL core --------
                    "state": prev_state,
                    "action": action,
                    "reward": reward,
                    "next_state": state,
                    "done": done,

                    # -------- Metadata --------
                    "termination_type": (
                        "success" if done and success else
                        "failure" if done else
                        None
                    )
                })

            prev_state = state
            prev_progress = progress
            completed_results.append(sub["result"])

        # ----------------------------------------------------
        # RETURN-TO-GO COMPUTATION
        # ----------------------------------------------------
        G = 0.0
        for t in reversed(episode_buffer):
            G = t["reward"] + GAMMA * G
            t["return_to_go"] = G

        transitions.extend(episode_buffer)
        episode_id += 1

    print(f"Prepared {len(transitions)} RL transitions.")
    return transitions

# ============================================================
# TRAIN / VAL SPLIT
# ============================================================

def split_dataset(samples, val_ratio=0.1, seed=42):
    random.seed(seed)
    random.shuffle(samples)
    split_idx = int(len(samples) * (1 - val_ratio))
    return samples[:split_idx], samples[split_idx:]

# ============================================================
# USAGE
# ============================================================

# ds must already be loaded in the notebook
# ds = [...]

rl_transitions = prepare_rl_dataset(ds)

train_data, val_data = split_dataset(rl_transitions)

with open("verigui_coordinator_rl_train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=2, ensure_ascii=False)

with open("verigui_coordinator_rl_val.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, indent=2, ensure_ascii=False)

print("✅ RL-ready dataset with task_id saved.")


Prepared 998 RL transitions.
✅ RL-ready dataset with task_id saved.
